In [6]:
import shutil
from collections import defaultdict
from pathlib import Path

In [8]:
# Move audio files into class-based subdirectories
SOURCE_DIR = Path("/home/div/vsCode/Audio_classifier_Model/data/urbansound8k")
DEST_DIR = SOURCE_DIR.parent / "urbansound8k_by_class"

CLASS_MAP = {
    "0": "air_conditioner",
    "1": "car_horn",
    "2": "children_playing",
    "3": "dog_bark",
    "4": "drilling",
    "5": "engine_idling",
    "6": "gun_shot",
    "7": "jackhammer",
    "8": "siren",
    "9": "street_music",
}

DEST_DIR.mkdir(exist_ok=True)
counts = defaultdict(int)

for audio_path in SOURCE_DIR.rglob("*.wav"):
    parts = audio_path.stem.split("-")
    if len(parts) < 2:
        continue
    class_id = parts[1]
    class_name = CLASS_MAP.get(class_id)
    if class_name is None:
        continue
    target_dir = DEST_DIR / class_name
    target_dir.mkdir(parents=True, exist_ok=True)
    target_path = target_dir / audio_path.name
    if target_path.exists():
        continue
    shutil.copy2(audio_path, target_path)  # switch to shutil.move(...) to relocate instead
    counts[class_name] += 1

for cls, total in sorted(counts.items()):
    print(f"{cls}: {total} files")

air_conditioner: 1000 files
car_horn: 429 files
children_playing: 1000 files
dog_bark: 1000 files
drilling: 1000 files
engine_idling: 1000 files
gun_shot: 374 files
jackhammer: 1000 files
siren: 929 files
street_music: 1000 files
